In [9]:
#|default_exp fastmcp_tools

In [10]:
#|export
"""
FastMCP tool-server for *Arena Buddy*
-------------------------------------

* Exposes every mobility / parking / weather helper as an **MCP tool**
* Runs over **Server-Sent Events** (SSE) – identical pattern to *ONE.py*
"""

from __future__ import annotations
import asyncio, datetime as dt
from pathlib import Path

# ── FastMCP server --------------------------------------------------------
from mcp.server.fastmcp import FastMCP

# ── Re-use helpers from the DataTalks library ----------------------------
from DataTalks.api   import geocode, osrm_car, ors_bike, osrm_walk, digitransit_pt
from DataTalks.arena import (
    AREENA,
    nearest_parking_ids,
    parking_status,
)

# ── very tiny weather helper (open-met Norway meteogram) ------------------
import httpx
async def weather_now(place: str = "Espoo,FI") -> dict:
    url = f"https://api.met.no/weatherapi/locationforecast/2.0/compact?lat={AREENA[0]}&lon={AREENA[1]}"
    async with httpx.AsyncClient(timeout=10, headers={"User-Agent": "arena-buddy"}) as cx:
        js = (await cx.get(url)).json()
    now = js["properties"]["timeseries"][0]
    temp = now["data"]["instant"]["details"]["air_temperature"]
    precip = now["data"].get("next_1_hours", {}).get("details", {}).get("precipitation_amount", 0)
    return {"temp_C": temp, "precip_mm": precip, "timestamp": now["time"]}

# ── Instantiate the server -----------------------------------------------
MCP = FastMCP("arena-tools", port=9001)

# ── ROUTE tools -----------------------------------------------------------
@MCP.tool()
async def route_walk(origin: str):
    "Walking Route from `origin` to Metro Areena (returns Route pydantic model)."
    src = await geocode(origin)
    return await osrm_walk(src, AREENA)

@MCP.tool()
async def route_bike(origin: str):
    "Cycling Route from `origin` to Metro Areena."
    src = await geocode(origin)
    return await ors_bike(src, AREENA)

@MCP.tool()
async def route_pt(origin: str):
    "Fastest public-transport itinerary from `origin` to Metro Areena."
    src = await geocode(origin)
    return await digitransit_pt(src, AREENA)

@MCP.tool()
async def route_car(origin: str):
    "Driving Route from `origin` to Metro Areena (fallback)."
    src = await geocode(origin)
    return await osrm_car(src, AREENA)

# ── PARKING tools ---------------------------------------------------------
@MCP.tool()
async def parking_live():
    "Current free / total CAR slots at the nearest P+R to Metro Areena."
    fid = (await nearest_parking_ids(*AREENA))["CAR"]
    return await parking_status(fid)

# ── WEATHER tool ----------------------------------------------------------
@MCP.tool()
async def arena_weather():
    "Current temperature °C and 1-h precipitation forecast at Metro Areena."
    return await weather_now()


#if __name__ == "__main__":
#    print("⚡ Arena-Buddy MCP tool-server running on http://0.0.0.0:9001/sse")
#    MCP.run(transport="sse")        # ← no host=



In [11]:
__all__ = ["MCP",
           "route_walk", "route_bike", "route_pt", "route_car",
           "parking_live", "arena_weather"]